In [ ]:
import torch
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx

edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index)


import networkx as nx
m_nx = nx.Graph(m.edges.tolist())
m_data = from_networkx(m_nx)
m_data2 = Data(x = m.vertices.x, edge_index = torch.cat((m.edges, m.edges[:, [1,0]])).t().contiguous() )


m_data

m_data2